In [1]:
from transformers import pipeline, set_seed
import gradio as gr
import datetime
from PIL import Image, ImageDraw, ImageFont


In [ ]:
set_seed(42)
generator = pipeline("text-generation", model="gpt2")  # or "EleutherAI/gpt-neo-1.3B"


In [3]:
def generate_story(prompt, genre="fantasy", max_length=200, num_stories=3):
    genre_prompt = f"This is a {genre} story.\n{prompt}"
    outputs = generator(genre_prompt, 
                        max_length=max_length, 
                        num_return_sequences=num_stories,
                        do_sample=True, 
                        top_k=50, 
                        top_p=0.95)
    return [o["generated_text"].strip() for o in outputs]


In [ ]:
def gradio_interface(prompt, genre, max_length, num_stories):
    stories = generate_story(prompt, genre, max_length, num_stories)
    story_text = "\n\n---\n\n".join([f"Option {i+1}:\n{story}" for i, story in enumerate(stories)])
    return story_text

gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.Textbox(lines=6, label="Story Prompt", placeholder="The knight entered the ruined temple..."),
        gr.Dropdown(["fantasy", "mystery", "sci-fi", "post-apocalyptic", "mythical", "adventure"], label="Genre"),
        gr.Slider(50, 400, value=200, label="Max Story Length"),
        gr.Slider(1, 5, value=3, label="Number of Story Options")
    ],
    outputs=gr.Textbox(label="Generated Story Options"),
    title="🧙 AI Dungeon Story Generator",
    description="Create fantasy/mystery/sci-fi stories using GPT-2",
    theme="default"
).launch(inline=True, share=True)

In [8]:
def save_story_image(text, filename="story_output.png"):
    # Create an image from text
    img = Image.new('RGB', (1000, 1400), color=(255, 255, 255))
    draw = ImageDraw.Draw(img)
    
    # Use default font
    font = ImageFont.load_default()
    
    # Wrap and draw text
    lines = text.split('\n')
    y_text = 10
    for line in lines:
        draw.text((10, y_text), line, font=font, fill=(0, 0, 0))
        y_text += 15
    
    img.save(filename)
    print(f"✅ Saved image as {filename}")
